# 关系型数据库映射(sqlalchemy)

对每一种数据库都用不同的操作虽然可以更细致的了解细节,但这种方式并不友好,为了做到数据库一站式服务,我们有了SQLAlchemy.

## 安装

用pip安装
    
    pip install SQLAlchemy
    

## 连接到数据库

现在的sqlalchemy支持主流的mysql,sqlite3,和postgresql

In [1]:
from sqlalchemy import *

In [2]:
db=create_engine("sqlite:///sqlalchemy.db")

SQLAlchemy实际上是两个库的包装:

+ 一个是SQL语言构造器,用来通过SQL语言操作表,
+ 另一个是ORM(对象关系映射器),也就是对象化数据库操作库.


## 操作数据库中的表操作



> 创建表

In [3]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [4]:
metadata = MetaData()

In [5]:
users = Table('users', metadata,
     Column('id', Integer, primary_key=True),
     Column('name', String),
     Column('fullname', String),
)

In [6]:
addresses = Table('addresses', metadata,
   Column('id', Integer, primary_key=True),
   Column('user_id', None, ForeignKey('users.id')),
   Column('email_address', String, nullable=False)
)

In [7]:
metadata.create_all(db)

上面的语句创建了两个表,他们用外键相连

> 插入数据

In [8]:
ins = users.insert().values(name=u"Jonathan",fullname=u"Jonathan Joestar")

In [9]:
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [10]:
ins.compile().params

{'fullname': u'Jonathan Joestar', 'name': u'Jonathan'}

上面的代码展示了插入数据操作实际对应的SQL语句

>> 执行

In [11]:
conn = db.connect()

In [12]:
conn

In [13]:
result = conn.execute(ins)

In [14]:
result

>> 动态绑定执行

In [15]:
ins = users.insert()
conn.execute(ins, name=u'Dio', fullname=u'Dio Brando')

>> 复数执行

In [16]:
conn.execute(addresses.insert(), [
    {'user_id': 1, 'email_address' : 'jojo@yahoo.com'},
    {'user_id': 1, 'email_address' : 'jojo@msn.com'},
    {'user_id': 2, 'email_address' : 'dio@www.org'},
    {'user_id': 2, 'email_address' : 'dio@aol.com'},
])


> 查询

In [17]:
from sqlalchemy.sql import select
s = select([users])
result = conn.execute(s)


In [18]:
for i in result:
    print(i)

(1, u'Jonathan', u'Jonathan Joestar')
(2, u'Dio', u'Dio Brando')


或者用fetchone方法提取

In [19]:
result = conn.execute(s)
row = result.fetchone()
row

(1, u'Jonathan', u'Jonathan Joestar')

>> 查询细节

In [20]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)
for row in result:
    print(row)

(u'Jonathan', u'Jonathan Joestar')
(u'Dio', u'Dio Brando')


>> 复杂查询

In [21]:
for row in conn.execute(select([users.c.fullname, addresses.c.email_address]).where(users.c.id == addresses.c.user_id)):
    print(row)

(u'Jonathan Joestar', u'jojo@yahoo.com')
(u'Jonathan Joestar', u'jojo@msn.com')
(u'Dio Brando', u'dio@www.org')
(u'Dio Brando', u'dio@aol.com')


>> and\_ or\_ not\_ like方法

逻辑运算在sqlalchemy中是这样的

    and_(expr1,expr2)
    
而like则是用于通配符操作

In [22]:
for row in conn.execute(select([users.c.fullname, 
                                addresses.c.email_address]).where(and_(users.c.id == addresses.c.user_id,
                                                                       addresses.c.email_address.like("%.com") ))):
    print(row)

(u'Jonathan Joestar', u'jojo@yahoo.com')
(u'Jonathan Joestar', u'jojo@msn.com')
(u'Dio Brando', u'dio@aol.com')


>> order,group,limit

有的时候我们希望排序找最优,sql中常用order,group和limit关键字,咋整呢?

In [23]:
conn.execute(select([users.c.name]).order_by(users.c.name)).fetchall()#排序,字母按字母表顺序

[(u'Dio',), (u'Jonathan',)]

In [24]:
conn.execute(select([users.c.name]).order_by(users.c.name.desc())).fetchall()#排序,字母按字母表逆序

[(u'Jonathan',), (u'Dio',)]

In [25]:
#分组
conn.execute(select([users.c.name,
                     func.count(addresses.c.id)]).select_from(users.join(addresses)).group_by(users.c.name).having(func.length(users.c.name) > 4)).fetchall()

[(u'Jonathan', 2)]

In [26]:
#限制数
conn.execute(select([users.c.name, 
                     addresses.c.email_address]).select_from(users.join(addresses)).limit(1).offset(1)).fetchall()

[(u'Jonathan', u'jojo@msn.com')]

> 别名 alias

有的时候表名字太长了或者不好记,我们会给他取个别名,这个时候就用alias了,这个和linux中的shell设置差不多

In [27]:
a1 = addresses.alias()
a2 = addresses.alias()
s = select([users]).where(and_(
users.c.id == a1.c.user_id,
users.c.id == a2.c.user_id,
a1.c.email_address == 'jojo@msn.com',
a2.c.email_address == 'jojo@yahoo.com'
))
conn.execute(s).fetchall()


[(1, u'Jonathan', u'Jonathan Joestar')]

> join 联合

数据库的联合操作算是个比较常见的操作了

In [28]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [29]:
s = select([users.c.fullname]).select_from(
users.join(addresses,
        addresses.c.email_address.like("%.com"))
)

In [30]:
conn.execute(s).fetchall()

[(u'Jonathan Joestar',),
 (u'Dio Brando',),
 (u'Jonathan Joestar',),
 (u'Dio Brando',),
 (u'Jonathan Joestar',),
 (u'Dio Brando',)]

> 更新表

In [31]:
stmt = select([addresses.c.email_address]).where(addresses.c.user_id == users.c.id).limit(1)
conn.execute(users.update().values(fullname=stmt))

In [32]:
conn.execute(select([users])).fetchall()

[(1, u'Jonathan', u'jojo@yahoo.com'), (2, u'Dio', u'dio@www.org')]

> 删除

In [33]:
conn.execute(users.delete())

In [34]:
conn.execute(addresses.delete())

## ORM

到目前为止都是些sql语句的封装而已,也许你觉得这没啥了不起的.但那些都是铺垫而已,正篇开始

所谓的ORM就是对象关系映射器,它的作用是让我们把数据当做一个对象来处理.我们来具体了解下

## 连接数据库

第一步依然是连接数据库,这个和之前是一样的

In [35]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ORMsql.db', echo=True)

> 创建一个类映射user的类

In [36]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()#为了演示看看内部怎么运行的我们用这个作为基类

from sqlalchemy import Column, Integer, String
#可以定义的时候直接用object做父类
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

In [37]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

In [38]:
Base.metadata.create_all(engine)#映射到数据库

2016-01-24 23:34:17,876 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2016-01-24 23:34:17,878 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2016-01-24 23:34:17,880 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:()


2016-01-24 23:34:17,884 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("users")


2016-01-24 23:34:17,886 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


> 创建一个实例

In [39]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')

> 创建一个会话

In [40]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)#bind是绑定数据库的作用,可以绑定也可以不绑以后绑定的话可以用Session.configure(bind=engine)

In [41]:
session = Session()#创建以歌会话的实例

> 把user对象添加到会话

In [42]:
session.add(ed_user)

> 复数的添加对象

In [43]:
session.add_all([
User(name='wendy', fullname='Wendy Williams', password='foobar'),
User(name='mary', fullname='Mary Contrary', password='xxg527'),
User(name='fred', fullname='Fred Flinstone', password='blah')])


In [44]:
session.commit()#讲会话提交到数据库

2016-01-24 23:37:43,614 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2016-01-24 23:37:43,628 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2016-01-24 23:37:43,630 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)


2016-01-24 23:37:43,644 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')


INFO:sqlalchemy.engine.base.Engine:('fred', 'Fred Flinstone', 'blah')


2016-01-24 23:37:43,648 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


> 在会话中查找

In [ ]:
our_user = session.query(User).filter_by(name='ed').first() 
our_user

In [ ]:
ed_user is our_user

In [ ]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

filter是过滤器,用来过滤那些不要的数据信息,可以在后面的对象中使用`like`,`in`,`~`,`is`,`and_`,`or_`,`match`等实现过滤

获取结果也可以通过all()获取结果的列表,first()获取第一个结果,count()获取结果的个数等

> 建立关系

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'     
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

这样就建立了两个对象间的联系了

注意,relationship中第一位参数是类名,back_populates后面是tablename